In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("datasets\\salaries.csv")
df.head()

,company,job,degree,salary_more_then_100k
0,google,sales executive,bachelors,0
1,google,sales executive,masters,0
2,google,business manager,bachelors,1
3,google,business manager,masters,1
4,google,computer programmer,bachelors,0


In [3]:
df.shape

(16, 4)

In [6]:
df.degree.unique()

array(['bachelors', 'masters'], dtype=object)

### Feature engineering- label and one hot encoding

In [7]:
df["degree_number"] = df["degree"].map({"bachelors":1, "masters":2})
df.head()

,company,job,degree,salary_more_then_100k,degree_number
0,google,sales executive,bachelors,0,1
1,google,sales executive,masters,0,2
2,google,business manager,bachelors,1,1
3,google,business manager,masters,1,2
4,google,computer programmer,bachelors,0,1


In [22]:
df.drop("degree", axis=1, inplace=True)
df.head()

,company,job,salary_more_then_100k,degree_number
0,google,sales executive,0,1
1,google,sales executive,0,2
2,google,business manager,1,1
3,google,business manager,1,2
4,google,computer programmer,0,1


In [23]:
df_encoded = pd.get_dummies(df, columns=["company", "job"], drop_first=True)

df_encoded.head()

,salary_more_then_100k,degree_number,company_facebook,company_google,job_computer programmer,job_sales executive
0,0,1,False,True,False,True
1,0,2,False,True,False,True
2,1,1,False,True,False,False
3,1,2,False,True,False,False
4,0,1,False,True,True,False


### Model training

In [25]:
from sklearn.tree import DecisionTreeClassifier

X = df_encoded.drop("salary_more_then_100k", axis=1)
y = df_encoded["salary_more_then_100k"]



In [26]:
model = DecisionTreeClassifier()
model.fit(X, y)

DecisionTreeClassifier()

In [27]:
from sklearn.metrics import classification_report

y_pred = model.predict(X)
clas_r = classification_report(y_pred=y_pred, y_true=y)
print(clas_r)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [28]:
from sklearn.tree import export_text

print(export_text(model, feature_names=list(X.columns)))

|--- company_facebook <= 0.50
|   |--- job_sales executive <= 0.50
|   |   |--- degree_number <= 1.50
|   |   |   |--- company_google <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- company_google >  0.50
|   |   |   |   |--- job_computer programmer <= 0.50
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- job_computer programmer >  0.50
|   |   |   |   |   |--- class: 0
|   |   |--- degree_number >  1.50
|   |   |   |--- class: 1
|   |--- job_sales executive >  0.50
|   |   |--- class: 0
|--- company_facebook >  0.50
|   |--- class: 1

